In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (20, 20)

In [ ]:
import os
import numpy as np
from PIL import Image
from io import BytesIO
import requests
from scipy.spatial.distance import jaccard
import pandas as pd
from elasticsearch import Elasticsearch, helpers
from sklearn.cluster import KMeans, AgglomerativeClustering
from tqdm import tqdm

# load features

In [ ]:
feature_vector_dir = "/Users/pimh/Desktop/feature_vectors/"

In [ ]:
# feature_vector_ids = np.random.choice(os.listdir(feature_vector_dir), 25_000)
feature_vector_ids = os.listdir(feature_vector_dir)

In [ ]:
feature_vector_paths = [
    os.path.join(feature_vector_dir, id) for id in feature_vector_ids
]

In [ ]:
feature_vectors = []
for path in feature_vector_paths:
    with open(path) as f:
        feature_vector = np.fromfile(f, dtype=np.float32)
        feature_vectors.append(feature_vector)

feature_vectors = np.stack(feature_vectors)

In [ ]:
feature_vectors.shape

# correlation matrix
take the absolute value because we just care about the _strength_ of the correlation, not its sign. strong clusters.

In [ ]:
corr_matrix = np.abs(np.corrcoef(feature_vectors.T))

In [ ]:
corr_matrix.shape

In [ ]:
sns.heatmap(corr_matrix[:500, :500])

# cluster correlated columns

In [ ]:
clusters = KMeans(n_clusters=256).fit(corr_matrix)

In [ ]:
clusters.labels_

In [ ]:
pd.Series(clusters.labels_).value_counts().reset_index(drop=True).plot();

# save them

In [ ]:
with open("data/column_labels.npy", "wb") as f:
    np.save(f, clusters.labels_)